In [1]:
from scipy.misc import imread, imshow, imsave
from glob import glob
import numpy as np
import os
from math import degrees, radians
import matplotlib as mpl 
from collections import Counter
import json
import sys
import re
import libspn as spn
np.set_printoptions(threshold=np.inf)

# Params

In [2]:
resolution = 0.02
num_angle_cells = 56
min_radius = 0.3
max_radius = 5
radius_factor = 1.15

In [3]:
angles = np.linspace(-180, 180, num_angle_cells+1)
print(angles)

r=min_radius
radiuses=[r]
v = 0.04
while r<max_radius:
    r = r+v
    radiuses.append(r)
    v*=radius_factor
    
radiuses = np.array(radiuses)
num_radius_cells = len(radiuses)-1
print(radiuses)
print(num_radius_cells)

[-180.         -173.57142857 -167.14285714 -160.71428571 -154.28571429
 -147.85714286 -141.42857143 -135.         -128.57142857 -122.14285714
 -115.71428571 -109.28571429 -102.85714286  -96.42857143  -90.
  -83.57142857  -77.14285714  -70.71428571  -64.28571429  -57.85714286
  -51.42857143  -45.          -38.57142857  -32.14285714  -25.71428571
  -19.28571429  -12.85714286   -6.42857143    0.            6.42857143
   12.85714286   19.28571429   25.71428571   32.14285714   38.57142857
   45.           51.42857143   57.85714286   64.28571429   70.71428571
   77.14285714   83.57142857   90.           96.42857143  102.85714286
  109.28571429  115.71428571  122.14285714  128.57142857  135.
  141.42857143  147.85714286  154.28571429  160.71428571  167.14285714
  173.57142857  180.        ]
[ 0.3         0.34        0.386       0.4389      0.499735    0.56969525
  0.65014954  0.74267197  0.84907276  0.97143368  1.11214873  1.27397104
  1.46006669  1.6740767   1.9201882   2.20321643  2.5286989

# Processing

In [4]:
def pixel_to_polar(scan_size, x, y):
    """Convert pixel coordinate to polar cell coordinate."""
    c_x = scan_size[0]//2
    c_y = scan_size[1]//2
    x = x - c_x
    y = y - c_y
    r = np.sqrt(x**2 + y**2) * resolution
    alpha = np.arctan2(-y, x) # Angles go clockwise with the -
    return (r, np.degrees(alpha))

pixel_to_polar((1000, 1000), 600, 500)

(2.0, 0.0)

In [5]:
def scan_to_polar(scan_image):
    ys,xs = np.meshgrid(np.arange(scan_image.shape[0])+0.5, np.arange(scan_image.shape[1])+0.5)
    rr, aa = pixel_to_polar(scan_image.shape, xs, ys)
    aa = np.digitize(aa, angles) - 1
    rr = np.digitize(rr, np.r_[0, radiuses]) - 1  # Additional cell for stuff near the robot
    polar_scan_elems = [[[] for _ in range(num_angle_cells)] for _ in range(num_radius_cells)]
    for x in range(scan_image.shape[0]):
        for y in range(scan_image.shape[1]):
            r = rr[x,y]
            a = aa[x,y]
            if r>0 and r<=num_radius_cells:
                polar_scan_elems[r-1][a].append(scan_image[x,y])
    for r in range(num_radius_cells):
        for a in range(num_angle_cells):
            vals=polar_scan_elems[r][a]
            free_count = sum(1 for i in vals if i>250)
            occupied_count = sum(1 for i in vals if i<10)
            unknown_count = len(vals) - free_count - occupied_count
            if not vals: # No elements!
                raise Exception("No elements in %s %s" % (r, a))
            if occupied_count/len(vals) > 0.01:        
                    val = 1
            elif free_count/len(vals) > 0.01:
                    val = 0
            else:
                val = -1
            polar_scan_elems[r][a]=val
    return np.array(polar_scan_elems)

# Plotting

In [6]:
def plot_polar_scan(polar_scan):
    a, r = np.meshgrid(np.radians(angles), radiuses)
    ax = plt.subplot(111, projection='polar')
    ax.set_theta_zero_location("S")
    ax.set_theta_direction(-1)
    cmap = mpl.colors.LinearSegmentedColormap.from_list('mycmap', [(0, 'gray'),
                                                        (0.5, 'white'),
                                                        (1, 'black')])

    ax.pcolormesh(a, r, polar_scan+1, cmap=cmap)
    plt.show()

# Load Scans

In [7]:
room_categories=['None', 'elevator', 'corridor', 'kitchen', '1p_office', '2p_office', 'mp_office', 'prof_office',
                 'anteroom', 'restroom', 'meeting_room', 'robot_lab', 'living_room', 'workshop', 'printer_area', 
                'large_meeting_room']

In [8]:
data_path = "/home/pronobis/Data/"

In [9]:
def process_sequence(seq_name, typ):
    scans_path = os.path.join(data_path, "virtual_scans_"+typ, seq_name)
    scan_pgms = sorted(glob(os.path.join(scans_path, "*.pgm"))) 
    proc_path = os.path.join(data_path, "proc_scans_"+typ, seq_name)
    floor_name = re.sub("_.*", "", seq_name)
    os.makedirs(proc_path, exist_ok=True)
    polar_scans = []
    for pgm in scan_pgms:
        fname = os.path.basename(pgm)
        tstamp = re.sub("_scan.*pgm", "", fname)
        # Load scan and annotations
        scan = imread(pgm)
        with open(os.path.join(scans_path, tstamp+"_annot.json")) as f:
            d = json.load(f)
        # Check if we want to keep this scan
        room_class = d['room_category']
        room_id = floor_name + "_" + d['room']
        # Store
        imsave(arr=scan, name=os.path.join(proc_path, tstamp + "_" + room_id + "_" + room_class + ".png"))
        polar_scan = scan_to_polar(scan)
        polar_scans.append([room_id, room_class, polar_scan.ravel()])
        sys.stdout.write('.')
    return polar_scans

In [10]:
import pickle
def save_sequence(seq, name, typ):
    polar_path = os.path.join(data_path, "polar_scans_"+typ)
    os.makedirs(polar_path, exist_ok=True)
    with open(os.path.join(polar_path, name), 'wb') as f:
        pickle.dump(seq, f)

In [11]:
sequences = sorted(os.listdir(os.path.join(data_path, "virtual_scans_real")))
print(sequences)

['floor4_cloudy_b_data', 'floor4_night_b_data', 'floor5_cloudy_b_data', 'floor5_night_b_data', 'floor6_cloudy_a2_data', 'floor6_night_a2_data', 'floor7_cloudy_b_data', 'floor7_night_b_data']


In [12]:
def do_sequence(seq_name):
    processed_seq = process_sequence(seq_name, typ='real')
    save_sequence(processed_seq, seq_name, typ='real')
#    processed_seq = process_sequence(seq_name, typ='testing')
#    save_sequence(processed_seq, seq_name, typ='testing')

In [13]:
do_sequence('floor4_cloudy_b_data')

.......................................................................................................................................................................................................................................................................................................................................

In [14]:
do_sequence('floor4_night_b_data')

.....................................................................................................................................................................................................................................................................................................................

In [15]:
do_sequence('floor5_cloudy_b_data')

................................................................................................................................................................................................................................................................................................................................

In [16]:
do_sequence('floor5_night_b_data')

.......................................................................................................................................................................................................................................................................................................................................................................

In [17]:
do_sequence('floor6_cloudy_a2_data')

........................................................................................................................................................

In [18]:
do_sequence('floor6_night_a2_data')

..............................................................................................................................................................................................

In [19]:
do_sequence('floor7_cloudy_b_data')

.........................................................................................................................................................................................................................................................................................................................................................................................................

In [20]:
do_sequence('floor7_night_b_data')

.............................................................................................................................................................................................................................................................................................................................................................................................

In [21]:
processed_seq = process_sequence('simulated', typ='simulated')
save_sequence(processed_seq, 'simulated', typ='simulated')

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................